In [1]:
import mne
import os
import time
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
from tqdm import tqdm   
import keras
import xml.etree.ElementTree as ET
from scipy.io import wavfile
from scipy.signal import find_peaks
import warnings

import tensorflow as tf

import sys
# sys.path.append(os.path.abspath("..")) 
sys.path.append("/var/data/apnea/SafeVision-Apnea/vggish/")

import vggish_input, vggish_params as params, vggish_slim
from vggish_slim import define_vggish_slim, load_vggish_slim_checkpoint

from scripts.lstm_input import get_mel_spectrograms, get_vggish_output
from scripts.vggish.vggish_model import load_vggish_checkpoint, VGGish  

2025-02-20 18:20:16.293492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-20 18:20:16.306329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-20 18:20:16.310367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 18:20:16.319003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
checkpoint_path = "/var/data/apnea/src/vggish/vggish_model.ckpt"  
pca_params_path = "/var/data/apnea/src/vggish/vggish_pca_params.npz"

APNEA_DIR = '/var/data/apnea/datasets/new_mic_dataset/1/'
NO_APNEA_DIR = '/var/data/apnea/datasets/new_mic_dataset/0/'
OUT_DIR = '/var/data/apnea/datasets/new_mic_dataset_spec/'

apnea_files = os.listdir(APNEA_DIR)
no_apnea_files = os.listdir(NO_APNEA_DIR)

# Извлечение весов из тензерфлоу 1

In [3]:
import tensorflow.compat.v1 as tf
#tf.compat.v1.enable_eager_execution()

tf.disable_eager_execution()

# Создаём граф TensorFlow 1.x
tf.reset_default_graph()

# Определяем модель VGGish в TF-Slim
features_tensor = tf.placeholder(tf.float32, shape=(None, params.NUM_FRAMES, params.NUM_BANDS), name='input_features')
embedding = define_vggish_slim(features_tensor, training=False)

# Загружаем чекпоинт
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path)

# Извлекаем веса из TF-Slim
variables = {v.name: sess.run(v) for v in tf.global_variables()}

sess.close()


/home/serv1/miniconda3/envs/last/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/serv1/miniconda3/envs/last/lib/python3.10/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
I0000 00:00:1740063318.639637 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740063318.643649 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

INFO:tensorflow:Restoring parameters from /var/data/apnea/src/vggish/vggish_model.ckpt


I0000 00:00:1740063318.987214 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740063318.988701 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740063318.990059 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740063318.991376 2153280 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [ ]:
np.savez("vggish_weights.npz", **variables)

✅ Переменные сохранены в vggish_weights.npz


# Загрузка весов и сохранение в формат keras

In [3]:
weights = np.load("vggish_weights.npz", allow_pickle=True)
variables = {key: weights[key] for key in weights.files}

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

def build_keras_vggish():
    inp = Input(shape=(params.NUM_FRAMES, params.NUM_BANDS, 1), name="input_features")

    x = Conv2D(64, (3, 3), activation="relu", padding="same", name="conv1")(inp)
    x = MaxPooling2D((2, 2), strides=2, padding="same", name="pool1")(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same", name="conv2")(x)
    x = MaxPooling2D((2, 2), strides=2, padding="same", name="pool2")(x)

    x = Conv2D(256, (3, 3), activation="relu", padding="same", name="conv3_1")(x)
    x = Conv2D(256, (3, 3), activation="relu", padding="same", name="conv3_2")(x)
    x = MaxPooling2D((2, 2), strides=2, padding="same", name="pool3")(x)

    x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv4_1")(x)
    x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv4_2")(x)
    x = MaxPooling2D((2, 2), strides=2, padding="same", name="pool4")(x)

    x = Flatten()(x)
    x = Dense(4096, activation="relu", name="fc1_1")(x)
    x = Dense(4096, activation="relu", name="fc1_2")(x)

    out = Dense(params.EMBEDDING_SIZE, name="fc2")(x)

    model = Model(inp, out, name="VGGish_Keras")
    return model


In [ ]:
keras_model = build_keras_vggish()

layer_mapping = {
    "vggish/conv1/weights:0": "conv1",
    "vggish/conv1/biases:0": "conv1",
    "vggish/conv2/weights:0": "conv2",
    "vggish/conv2/biases:0": "conv2",
    "vggish/conv3/conv3_1/weights:0": "conv3_1",
    "vggish/conv3/conv3_1/biases:0": "conv3_1",
    "vggish/conv3/conv3_2/weights:0": "conv3_2",
    "vggish/conv3/conv3_2/biases:0": "conv3_2",
    "vggish/conv4/conv4_1/weights:0": "conv4_1",
    "vggish/conv4/conv4_1/biases:0": "conv4_1",
    "vggish/conv4/conv4_2/weights:0": "conv4_2",
    "vggish/conv4/conv4_2/biases:0": "conv4_2",
    "vggish/fc1/fc1_1/weights:0": "fc1_1",
    "vggish/fc1/fc1_1/biases:0": "fc1_1",
    "vggish/fc1/fc1_2/weights:0": "fc1_2",
    "vggish/fc1/fc1_2/biases:0": "fc1_2",
    "vggish/fc2/weights:0": "fc2",
    "vggish/fc2/biases:0": "fc2",
}

for tf_var, keras_layer in layer_mapping.items():
    if "weights" in tf_var:
        kernel = variables[tf_var]
        bias_var = tf_var.replace("weights", "biases")  # Поиск смещения
        bias = variables[bias_var] if bias_var in variables else None
        if bias is not None:
            keras_model.get_layer(keras_layer).set_weights([kernel, bias])
        else:
            keras_model.get_layer(keras_layer).set_weights([kernel])  # Если вдруг смещения нет
    elif "biases" in tf_var:
        pass  # Смещение уже загружено


In [6]:
keras_model.save("vggish_model.keras")

# Тестирование

In [4]:
vggish = keras.models.load_model('/var/data/apnea/SafeVision-Apnea/src/vggish_model.keras')

I0000 00:00:1740064832.623233   10346 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740064832.624979   10346 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740064832.675372   10346 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740064832.676824   10346 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [7]:
vggish.summary()

Model: "VGGish_Keras"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)     │ (None, 96, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 96, 64, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 48, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 48, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 24, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_1 (Conv2D)                │ (None, 24, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_2 (Conv2D)                │ (None, 24, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 12, 8, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_1 (Conv2D)                │ (None, 12, 8, 512)     │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_2 (Conv2D)                │ (None, 12, 8, 512)     │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 6, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1_1 (Dense)                   │ (None, 4096)           │    50,335,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1_2 (Dense)                   │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 128)            │       524,416 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,141,184 (275.20 MB)

 Trainable params: 72,141,184 (275.20 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.models import load_model, Model

intermediate_model = Model(inputs=vggish.input, outputs=vggish.get_layer('pool4').output)